In [1]:
from __future__ import division
from __future__ import print_function

import numpy as np
from numpy.random import rand
import matplotlib.pyplot as plt
import os

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow.keras as keras

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers

import gc
from numba import cuda
from numba import jit

In [2]:
# This is how the triagle lattice data is generated. You may find it helpful to generate some 
# of your own data
class Ising_tri():
    ''' Simulating the Ising model '''  
    def __init__(self, size, temp):
        self.temp = temp
        self.N = int(size)
    ## monte carlo moves
    
    def mcmove(self, config, N, beta):
        ''' This is to execute the monte carlo moves using 
        Metropolis algorithm such that detailed
        balance condition is satisified'''
        for i in range(N):
            for j in range(N):            
                    a = np.random.randint(0, N) # select a row
                    b = np.random.randint(0, N) # select a column
                    s =  config[a, b] # current state at (a, b)
                    if a%2:
                        nb = config[(a+1)%N,b] +config[(a+1)%N,(b+1)%N] + config[a,(b+1)%N] + \
                        config[(a-1)%N,b] + config[(a-1)%N,(b+1)%N] + config[a,(b-1)%N]
                    else:
                        nb = config[(a+1)%N,b] +config[(a+1)%N,(b-1)%N] + config[a,(b+1)%N] + \
                        config[(a-1)%N,b] + config[(a-1)%N,(b-1)%N] + config[a,(b-1)%N]
                    
                    
                    cost = 2*s*nb
                    if cost < 0:	
                        s *= -1
                    elif rand() < np.exp(-cost*beta):
                        s *= -1
                    config[a, b] = s
        return config
    
    def simulate(self):   
        ''' This module simulates the Ising model'''
        config = 2*np.random.randint(2, size=(self.N,self.N))-1   
        msrmnt = 81
        for i in range(msrmnt):
            self.mcmove(config, self.N, 1.0/self.temp)
        return config

class Ising_sq():
    ''' Simulating the Ising model '''  
    def __init__(self, size, temp):
        self.temp = temp
        self.N = int(size)
    ## monte carlo moves

    def mcmove(self, config, N, beta):
        ''' This is to execute the monte carlo moves using 
        Metropolis algorithm such that detailed
        balance condition is satisified'''
        for i in range(N):
            for j in range(N):            
                    a = np.random.randint(0, N) # select a row
                    b = np.random.randint(0, N) # select a column
                    s =  config[a, b] # current state at (a, b)
                    # periodic boundary condition imposed
                    nb = config[(a+1)%N,b] + config[a,(b+1)%N] + config[(a-1)%N,b] + config[a,(b-1)%N]
                    cost = 2*s*nb
                    if cost < 0:
                        s *= -1
                    elif rand() < np.exp(-cost*beta):
                        s *= -1
                    config[a, b] = s
        return config
    
    def simulate(self):   
        ''' This module simulates the Ising model'''
        config = 2*np.random.randint(2, size=(self.N,self.N))-1   
        times = 100
        for i in range(times):
            self.mcmove(config, self.N, 1.0/self.temp)
        return config

You can import 4-temp data for square and triangular lattices as follows

In [3]:
N = 250
nx, ny = 32, 32

Xsq = np.ndarray((4*N,nx,ny,1))
ysq = np.ndarray(4*N)

for i in np.arange(N):
    Xsq[i + 0*N] = np.loadtxt("./square_T1/square_T1/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
    ysq[i + 0*N] = 0
    Xsq[i + 1*N] = np.loadtxt("./square_T2/square_T2/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
    ysq[i + 1*N] = 1
    Xsq[i + 2*N] = np.loadtxt("./square_T3/square_T3/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
    ysq[i + 2*N] = 2
    Xsq[i + 3*N] = np.loadtxt("./square_T4/square_T4/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
    ysq[i + 3*N] = 3

Xsq_train, Xsq_test, ysq_train, ysq_test = train_test_split(Xsq, ysq, test_size=0.2, random_state=0)

In [4]:
N = 250
nx, ny = 32, 32

Xtri = np.ndarray((4*N,nx,ny,1))
ytri = np.ndarray(4*N)

for i in np.arange(N):
    Xtri[i + 0*N] = np.loadtxt("./triangle_T1/triangle_T1/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
    ytri[i + 0*N] = 0
    Xtri[i + 1*N] = np.loadtxt("./triangle_T2/triangle_T2/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
    ytri[i + 1*N] = 1
    Xtri[i + 2*N] = np.loadtxt("./triangle_T3/triangle_T3/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
    ytri[i + 2*N] = 2
    Xtri[i + 3*N] = np.loadtxt("./triangle_T4/triangle_T4/{:03d}".format(i), delimiter=",").reshape(nx,ny,1)
    ytri[i + 3*N] = 3

Xtri_train, Xtri_test, ytri_train, ytri_test = train_test_split(Xtri, ytri, test_size=0.2, random_state=0)

Make sure you know the shape of data.

In [5]:
print("Shape of training data:")
print(Xsq_train.shape, Xtri_train.shape)
print(ysq_train.shape, ytri_train.shape)
print("Shape of test data:")
print(Xsq_test.shape, Xtri_test.shape)
print(ysq_test.shape, ytri_test.shape)

Shape of training data:
(800, 32, 32, 1) (800, 32, 32, 1)
(800,) (800,)
Shape of test data:
(200, 32, 32, 1) (200, 32, 32, 1)
(200,) (200,)


### (a) Train a fully connected neural network to do the classification on both datasets. Then, train  a  convolutional  neural  network  to  do  the  classification,  on  both datasets.   Make  a  table  of  your  performance  numbers  for  both  models  and  upload  these  numbers.   This,  together  with  your code,  should be uploaded to the course website when you turn in your homework.

The temperatures for square lattice are $T = 1.5, 2.1, 2.4, 3.5$. $T = 2.5, 3.2, 3.8, 5$ for triangle lattice.


Solution to (a):

In [6]:
class small_FNN:
    def __init__(self):
        model = self
    
    @staticmethod
    def build(width, height, depth, num_classes, channels_first=False):
        model = Sequential()

        if channels_first == False:
            inputShape = (height, width, depth)
            chanDim = -1
        elif channels_first == True:
            inputShape = (depth, height, width)
            chanDim = 1
            
        model.add(Flatten())
        
        model.add(Dense(256,  activation='relu'))
        model.add(Dropout(0.2))
        model.add(BatchNormalization())
        
        model.add(Dense(128,  activation='relu'))
        model.add(Dropout(0.2))
        model.add(BatchNormalization())
        
        model.add(Dense(64, activation='relu'))
        model.add(Dropout(0.2))
        model.add(BatchNormalization())
        
        model.add(Dense(32, activation='relu'))
        model.add(Dropout(0.2))
        model.add(BatchNormalization())
        
        model.add(Dense(16, activation='relu'))
        model.add(Dropout(0.2))
        model.add(BatchNormalization())
        
        model.add(Dense(num_classes, activation="softmax"))
        
        return model

In [21]:
def train_model(input_model, train_data, train_lbls, test_data, 
                test_lbls, num_classes, input_shape, hyperparams):
    # Ensure data is shaped properly, assumes channels last set up
    x_train = train_data
    x_test = test_data
    
    # Create categorical labels
    y_train = keras.utils.to_categorical(train_lbls, num_classes)
    y_test = keras.utils.to_categorical(test_lbls, num_classes)
     
    # Set hyperparameters
    INIT_LR = hyperparams[0]# learning rate
    EPOCHS = hyperparams[1] # number of epochs
    BS = hyperparams[2] # batch size
    OPT = optimizers.Adagrad(lr=INIT_LR) # optimizing function
    
    model = input_model
    
    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer=OPT, metrics=['accuracy'])
    
    
    H = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=EPOCHS,
                  batch_size=BS)
    
    return H, model
    

In [24]:
input_shape = (32, 32, 1)
num_classes = 4
fnn_model_sq = small_FNN.build(width=input_shape[0], height=input_shape[1], depth=input_shape[2],
                   num_classes=num_classes)
fnn_model_tri = small_FNN.build(width=input_shape[0], height=input_shape[1], depth=input_shape[2],
                   num_classes=num_classes)

# hyperparams should be a tuple of: INIT_LR, EPOCHS, BS
FNN_hyperparams = (0.01, 50, 32)
H_sq_FNN, sq_FNN_model = train_model(fnn_model_sq, Xsq_train, ysq_train, Xsq_test, ysq_test, num_classes, input_shape, FNN_hyperparams)
H_tri_FNN, tri_FNN_model = train_model(fnn_model_tri, Xtri_train, ytri_train, Xtri_test, ytri_test, num_classes, input_shape, FNN_hyperparams)

Train on 800 samples, validate on 200 samples
Epoch 1/50
800/800 [==============================] - 1s 1ms/sample - loss: 1.8462 - accuracy: 0.2475 - val_loss: 1.3925 - val_accuracy: 0.2950
Epoch 2/50
800/800 [==============================] - 0s 111us/sample - loss: 1.5707 - accuracy: 0.2988 - val_loss: 1.4008 - val_accuracy: 0.2850
Epoch 3/50
800/800 [==============================] - 0s 109us/sample - loss: 1.4859 - accuracy: 0.3338 - val_loss: 1.3785 - val_accuracy: 0.3000
Epoch 4/50
800/800 [==============================] - 0s 108us/sample - loss: 1.3184 - accuracy: 0.4025 - val_loss: 1.3373 - val_accuracy: 0.3100
Epoch 5/50
800/800 [==============================] - 0s 102us/sample - loss: 1.3156 - accuracy: 0.4150 - val_loss: 1.3015 - val_accuracy: 0.3350
Epoch 6/50
800/800 [==============================] - 0s 105us/sample - loss: 1.2500 - accuracy: 0.4162 - val_loss: 1.2567 - val_accuracy: 0.3600
Epoch 7/50
800/800 [==============================] - 0s 110us/sample - loss: 1.

Epoch 6/50
800/800 [==============================] - 0s 105us/sample - loss: 1.2634 - accuracy: 0.4263 - val_loss: 1.3042 - val_accuracy: 0.2950
Epoch 7/50
800/800 [==============================] - 0s 102us/sample - loss: 1.1990 - accuracy: 0.4700 - val_loss: 1.2684 - val_accuracy: 0.3300
Epoch 8/50
800/800 [==============================] - 0s 115us/sample - loss: 1.1816 - accuracy: 0.4737 - val_loss: 1.2466 - val_accuracy: 0.3500
Epoch 9/50
800/800 [==============================] - 0s 130us/sample - loss: 1.1338 - accuracy: 0.4700 - val_loss: 1.2009 - val_accuracy: 0.3750
Epoch 10/50
800/800 [==============================] - 0s 135us/sample - loss: 1.0609 - accuracy: 0.5312 - val_loss: 1.1917 - val_accuracy: 0.3900
Epoch 11/50
800/800 [==============================] - 0s 138us/sample - loss: 1.0229 - accuracy: 0.5450 - val_loss: 1.1715 - val_accuracy: 0.3900
Epoch 12/50
800/800 [==============================] - 0s 136us/sample - loss: 0.9682 - accuracy: 0.5888 - val_loss: 1.146

### (b) Train a convolutional neural network to do the classification, on both datasets. Make a table of your performance numbers for (a) and (b). 
Try to optimize the performance of your models and compare the result.

solution to (b):

In [13]:
class SimpleCNN:
    def __init__(self):
        model = self
    
    @staticmethod
    def build(width, height, depth, num_classes, channels_first=False):
        # initialize the model along with the input shape to be
        # "channels last" and the channels dimension itself
        model = Sequential()
        
        if channels_first == False:
            inputShape = (height, width, depth)
            chanDim = -1
        elif channels_first == True:
            inputShape = (depth, height, width)
            chanDim = 1
            
        model.add(Conv2D(32, (5, 5), activation='relu', input_shape=inputShape))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(Dropout(0.25))
                  
        model.add(Flatten())
        model.add(Dropout(0.4))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(num_classes, activation='softmax'))

        # return the constructed network architecture
        return model

In [25]:
input_shape = (32, 32, 1)
num_classes = 4
cnn_model_sq = SimpleCNN.build(width=input_shape[0], height=input_shape[1], depth=input_shape[2],
                   num_classes=num_classes)
cnn_model_tri = SimpleCNN.build(width=input_shape[0], height=input_shape[1], depth=input_shape[2],
                   num_classes=num_classes)

CNN_hyperparams = (0.01, 50, 64)
H_sq_CNN, sq_CNN_model = train_model(cnn_model_sq, Xsq_train, ysq_train, Xsq_test, ysq_test, num_classes, input_shape, CNN_hyperparams)
H_tri_CNN, tri_CNN_model = train_model(cnn_model_tri, Xtri_train, ytri_train, Xtri_test, ytri_test, num_classes, input_shape, CNN_hyperparams)

Train on 800 samples, validate on 200 samples
Epoch 1/50
800/800 [==============================] - 0s 482us/sample - loss: 1.3340 - accuracy: 0.3525 - val_loss: 1.2205 - val_accuracy: 0.3900
Epoch 2/50
800/800 [==============================] - 0s 83us/sample - loss: 1.1370 - accuracy: 0.5325 - val_loss: 0.9143 - val_accuracy: 0.6900
Epoch 3/50
800/800 [==============================] - 0s 85us/sample - loss: 0.9296 - accuracy: 0.5962 - val_loss: 0.7645 - val_accuracy: 0.7200
Epoch 4/50
800/800 [==============================] - 0s 86us/sample - loss: 0.8056 - accuracy: 0.6750 - val_loss: 0.6541 - val_accuracy: 0.7350
Epoch 5/50
800/800 [==============================] - 0s 86us/sample - loss: 0.5799 - accuracy: 0.7563 - val_loss: 0.9467 - val_accuracy: 0.5250
Epoch 6/50
800/800 [==============================] - 0s 84us/sample - loss: 0.6365 - accuracy: 0.7125 - val_loss: 0.7360 - val_accuracy: 0.5800
Epoch 7/50
800/800 [==============================] - 0s 75us/sample - loss: 0.4013

800/800 [==============================] - 0s 85us/sample - loss: 0.7681 - accuracy: 0.7025 - val_loss: 0.8330 - val_accuracy: 0.4900
Epoch 7/50
800/800 [==============================] - 0s 84us/sample - loss: 0.6390 - accuracy: 0.7163 - val_loss: 0.7733 - val_accuracy: 0.6300
Epoch 8/50
800/800 [==============================] - 0s 81us/sample - loss: 0.5582 - accuracy: 0.7387 - val_loss: 0.4406 - val_accuracy: 0.8550
Epoch 9/50
800/800 [==============================] - 0s 79us/sample - loss: 0.4732 - accuracy: 0.7975 - val_loss: 1.1859 - val_accuracy: 0.4650
Epoch 10/50
800/800 [==============================] - 0s 81us/sample - loss: 0.7531 - accuracy: 0.7150 - val_loss: 0.4038 - val_accuracy: 0.9050
Epoch 11/50
800/800 [==============================] - 0s 74us/sample - loss: 0.4569 - accuracy: 0.7987 - val_loss: 0.4363 - val_accuracy: 0.8650
Epoch 12/50
800/800 [==============================] - 0s 77us/sample - loss: 0.4001 - accuracy: 0.8238 - val_loss: 0.4184 - val_accuracy: 

### (c) We have provided a test set of 10 spins configurations for each of the two problems. Each of the spin configurations is not necessarily at the temperatures of the training sets. Calculate your best estimate of the temperatures of these spin configuration. Upload your results to Kaggle.
[Hint: A direct fingerprint of temperature is the distribution of spin up
and down, because you can image that the spins fluctuate more violently
at higher temperature. Although the mothod you use in homework 2 can also work, you may be interested in trying to take distribution into account when you
build the model to estimate temperature and see if you can make use of this extra information. This may help you win the
kaggle. It is totally fine if you find that the information of distribution is not helpful. Note also that a CNN kind-of does this. One possibility is that you may want a CNN that captures enough distribution information.]

Solution to (c)

In [27]:
cwd = str(os.getcwd())
sq_sim_data_path = cwd+"/sq_sim_data.npy"
tri_sim_data_path = cwd+"/tri_sim_data.npy"

temps_vec = np.linspace(0.01,15,num=1500)

"""
sq_sim_data = []
tri_sim_data = []
sim_data_temp = []

for temp in temps_vec:    
    sq_ising_simu = Ising_sq(32, temp)
    tri_ising_simu = Ising_tri(32, temp)
    sq_img = sq_ising_simu.simulate()
    tri_img = tri_ising_simu.simulate()
    
    sq_sim_data.append(sq_img)
    tri_sim_data.append(tri_img)
    
cwd = str(os.getcwd())
sim_data_path_2 = cwd+"/sim_data_2.npy"
sim_data_temp_path_2 = cwd+"/sim_data_temp_2.npy"
sim_data_tot_lv_mean_path = cwd+"/sim_data_tot_lv_mean.npy"

np.save(sq_sim_data_path, np.asarray(sq_sim_data))
np.save(tri_sim_data_path, np.asarray(tri_sim_data))
"""

'\nsq_sim_data = []\ntri_sim_data = []\nsim_data_temp = []\n\nfor temp in temps_vec:    \n    sq_ising_simu = Ising_sq(32, temp)\n    tri_ising_simu = Ising_tri(32, temp)\n    sq_img = sq_ising_simu.simulate()\n    tri_img = tri_ising_simu.simulate()\n    \n    sq_sim_data.append(sq_img)\n    sq_sim_data.append(sq_img)\n    \ncwd = str(os.getcwd())\nsim_data_path_2 = cwd+"/sim_data_2.npy"\nsim_data_temp_path_2 = cwd+"/sim_data_temp_2.npy"\nsim_data_tot_lv_mean_path = cwd+"/sim_data_tot_lv_mean.npy"\n\nnp.save(sq_sim_data_path, np.asarray(sq_sim_data))\nnp.save(tri_sim_data_path, np.asarray(tri_sim_data))\n'

In [62]:
# Get the data ready to be embedded
sq_sim_data = np.load(sq_sim_data_path)[0:1500,:,:].reshape(1500,32,32,-1)
tri_sim_data = np.load(tri_sim_data_path).reshape(1500,32,32,-1)

In [63]:
sq_sim_data, tri_sim_data = np.repeat(sq_sim_data, 5, 1), np.repeat(tri_sim_data, 5, 1)
sq_sim_data, tri_sim_data = np.repeat(sq_sim_data, 5, 2), np.repeat(tri_sim_data, 5, 2)
sq_sim_data, tri_sim_data = np.repeat(sq_sim_data, 3, 3), np.repeat(tri_sim_data, 3, 3)

In [64]:
tri_sim_data.shape

(1500, 160, 160, 0)

In [65]:
# Get pre-trained MobileNetV2 
with tf.device('/CPU:0'):
    base_model = tf.keras.applications.MobileNetV2(input_shape=(160,160,3),
                                                   include_top=False,
                                                   weights='imagenet')
    base_model.trainable = False

In [66]:
# Get the embedded data
global_avg_layer = tf.keras.layers.GlobalAveragePooling2D()

sq_sim_data_emb = global_avg_layer(base_model.predict(sq_sim_data))
sq_sim_data_emb = sq_sim_data_emb.numpy()/sq_sim_data_emb.numpy().max()

#tri_sim_data_emb = global_avg_layer(base_model.predict(tri_sim_data))
#tri_sim_data_emb = tri_sim_data_emb.numpy()/tri_sim_data_emb.numpy().max()

In [67]:
# Get train/test splits
sq_x_train, sq_x_test, sq_y_train, sq_y_test = train_test_split(sq_sim_data_emb, temps_vec, test_size=0.2, random_state=0)
#tri_x_train, tri_x_test, tri_y_train, tri_y_test = train_test_split(tri_sim_data_emb, temps_vec, test_size=0.2, random_state=0)

In [76]:
# Define the model
class RegressionHead:
    def __init__(self):
        model = self
    
    @staticmethod
    def build(channels_first=False):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Dense(128, activation='relu'))
        model.add(tf.keras.layers.Dropout(0.25))
        model.add(tf.keras.layers.Dense(32, activation='relu'))
        model.add(tf.keras.layers.Dropout(0.25))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dense(num_classes, activation='linear'))
        
        return model

In [77]:
INIT_LR = 0.01# learning rate
EPOCHS = 50 # number of epochs
BS = 32 # batch size
OPT = optimizers.Adagrad(lr=INIT_LR) # optimizing function
LOSS = keras.losses.MeanSquaredError()

sq_reg_model = RegressionHead.build()
sq_reg_model.compile(optimizer=OPT, loss=LOSS, metrics=['MeanSquaredError'])
sq_reg_H = sq_reg_model.fit(sq_x_train, sq_y_train, validation_data=(sq_x_test, sq_y_test), epochs=EPOCHS,
                  batch_size=BS)

Train on 1200 samples, validate on 300 samples
Epoch 1/50
1200/1200 [==============================] - 1s 519us/sample - loss: 56.0420 - MeanSquaredError: 56.0420 - val_loss: 59.5491 - val_MeanSquaredError: 59.5491
Epoch 2/50
1200/1200 [==============================] - 0s 72us/sample - loss: 43.7723 - MeanSquaredError: 43.7723 - val_loss: 49.8098 - val_MeanSquaredError: 49.8098
Epoch 3/50
1200/1200 [==============================] - 0s 72us/sample - loss: 35.8806 - MeanSquaredError: 35.8806 - val_loss: 40.3151 - val_MeanSquaredError: 40.3151
Epoch 4/50
1200/1200 [==============================] - 0s 80us/sample - loss: 28.7519 - MeanSquaredError: 28.7519 - val_loss: 35.4987 - val_MeanSquaredError: 35.4987
Epoch 5/50
1200/1200 [==============================] - 0s 80us/sample - loss: 22.7629 - MeanSquaredError: 22.7629 - val_loss: 31.1740 - val_MeanSquaredError: 31.1740
Epoch 6/50
1200/1200 [==============================] - 0s 80us/sample - loss: 18.0515 - MeanSquaredError: 18.0515 - 

1200/1200 [==============================] - 0s 80us/sample - loss: 1.9682 - MeanSquaredError: 1.9682 - val_loss: 0.9610 - val_MeanSquaredError: 0.9610


In [ ]:
INIT_LR = 0.01# learning rate
EPOCHS = 50 # number of epochs
BS = 64 # batch size
OPT = optimizers.Adagrad(lr=INIT_LR) # optimizing function

tri_reg_model = RegressionHead.build()
tri_reg_model.compile(optimizer=OPT, loss='MeanSquaredError', metrics=['MeanSquaredError'])
tri_reg_H = tri_reg_model.fit(tri_x_train, tri_y_train, validation_data=(tri_x_test, tri_y_test), epochs=EPOCHS,
                  batch_size=BS)

### (d) *Transfer Learning*.  
As we emphasize in class, one can freeze the training of the bottom layers of a network and retrain the top part of the network to adopt to a new situation. Use your CNN that you trained on the squarelattice data to do transfer learning on the triangular lattice data.  How does the performance compare to that of the direct methods?  Add the performance numbers for transfer learning in your table from Part (a). Note that the training time and number of training examples needed for transfer learning is far less than that for the direct  optimization. For  example,  is  50  triangle  example  sufficient  for the re-training process?  Use your transfer learning result to predict the transition temperature of triangle lattice Ising model, as demonstrated in this [Nature Physics](https://www-nature-com.ezp-prod1.hul.harvard.edu/articles/nphys4035.pdf) publication.

As a guideline, you may like to just change the last `Dense` layer with `softmax` activation when you do the transfer learning. Other choices are also OK.

Solution to (d):

In [ ]:
base_model = sq_CNN_model

In [ ]:
base_model.summary()

In [ ]:
base_layers = base_model.layers[0:5]
trainable_layers = [
     Flatten(),
     Dropout(0.25),
     Dense(32, activation='relu'),
     Dense(num_classes, activation='softmax')
]

In [ ]:
trans_model = keras.Sequential(base_layers+trainable_layers)
trans_model.summary()

In [ ]:
input_shape = (32, 32, 1)
num_classes = 4
hyperparams = (0.01, 25, 32)
H_trans, trained_trans_model = train_model(trans_model, Xtri_train, ytri_train, Xtri_test, ytri_test, num_classes, input_shape, hyperparams)